In [1]:
import pandas as pd
import numpy as np
from collections import Counter

# dataset is accessible at https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt (https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz)
df = pd.read_csv('amazon_reviews_us_Digital_Software_v1_00.tsv', sep='\t', header=None, on_bad_lines='skip')

print(df.head())

            0            1               2           3               4   \
0  marketplace  customer_id       review_id  product_id  product_parent   
1           US     17747349  R2EI7QLPK4LF7U  B00U7LCE6A       106182406   
2           US     10956619  R1W5OMFK1Q3I3O  B00HRJMOM4       162269768   
3           US     13132245   RPZWSYWRP92GI  B00P31G9PQ       831433899   
4           US     35717248  R2WQWM04XHD9US  B00FGDEPDY       991059534   

                                           5                 6            7   \
0                               product_title  product_category  star_rating   
1                    CCleaner Free [Download]  Digital_Software            4   
2          ResumeMaker Professional Deluxe 18  Digital_Software            3   
3                   Amazon Drive Desktop [PC]  Digital_Software            1   
4  Norton Internet Security 1 User 3 Licenses  Digital_Software            5   

              8            9     10                 11              

c:\Users\Frederico\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3397: DtypeWarning: Columns (1,4,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101837 entries, 0 to 101836
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       101837 non-null  object
 1   1       101837 non-null  object
 2   2       101837 non-null  object
 3   3       101837 non-null  object
 4   4       101837 non-null  object
 5   5       101837 non-null  object
 6   6       101837 non-null  object
 7   7       101837 non-null  object
 8   8       101837 non-null  object
 9   9       101837 non-null  object
 10  10      101837 non-null  object
 11  11      101837 non-null  object
 12  12      101837 non-null  object
 13  13      101837 non-null  object
 14  14      101832 non-null  object
dtypes: object(15)
memory usage: 11.7+ MB


In [3]:
df.columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 
            'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']

print(df.head())

   marketplace  customer_id       review_id  product_id  product_parent  \
0  marketplace  customer_id       review_id  product_id  product_parent   
1           US     17747349  R2EI7QLPK4LF7U  B00U7LCE6A       106182406   
2           US     10956619  R1W5OMFK1Q3I3O  B00HRJMOM4       162269768   
3           US     13132245   RPZWSYWRP92GI  B00P31G9PQ       831433899   
4           US     35717248  R2WQWM04XHD9US  B00FGDEPDY       991059534   

                                product_title  product_category  star_rating  \
0                               product_title  product_category  star_rating   
1                    CCleaner Free [Download]  Digital_Software            4   
2          ResumeMaker Professional Deluxe 18  Digital_Software            3   
3                   Amazon Drive Desktop [PC]  Digital_Software            1   
4  Norton Internet Security 1 User 3 Licenses  Digital_Software            5   

   helpful_votes  total_votes  vine  verified_purchase     review_he

In [4]:
print(df['review_body'][1:10])
print(df['star_rating'][1:10])  

1                                       So far so good
2                        Needs a little more work.....
3                                       Please cancel.
4                                   Works as Expected!
5    I've had Webroot for a few years. It expired a...
6    EXCELLENT software !!!!!  Don't need to do any...
7    The variations created by Win10 induced this p...
8    Horrible!  Would not upgrade previous version ...
9                                      Waste of time .
Name: review_body, dtype: object
1    4
2    3
3    1
4    5
5    4
6    5
7    1
8    1
9    1
Name: star_rating, dtype: object


## Sentiment analysis

In [5]:
all_reviews = df['review_body'][1:]
all_ratings = df['star_rating'][1:]

In [6]:
reviews = list()

for review in all_reviews:
    reviews.append(review)

print(reviews[:10])

['So far so good', 'Needs a little more work.....', 'Please cancel.', 'Works as Expected!', "I've had Webroot for a few years. It expired and I decided to purchase a renewal on Amazon. I went through hell trying to uninstall the expired version in order to install the new.  I called Webroot and had their representative remote into my computer at his request. He was clueless as a bad joke and consumed 29 minutes and 57 seconds of my time forever.  He initially told me it wasn't compatible with Windows 10, but I finally managed to convince him that it is indeed compatible with Windows 10 as it was working on my computer before it expired and also I showed him a review on Amazon to convince him that it works on Windows 10. Finally, he offered to connect me with a senior consultant for over 100 dollars. I declined and told him I'd fix the issue myself. This guy was less helpful than a severed limb.  After spending some time on Google, the issue is now fixed. Webroot should just get rid of 

In [7]:
#from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

# Set the model name
MODEL_NAME = 'bert-base-cased'

# Build a BERT based tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

sample_text = "originally gave this a 2 star but I think it only deserves 1. I simply can't wrap my head around the following. picture this, you add a task to your list. the reminder feature is on.you want to change the time on the reminder feature so you click it."
               
tokens = tokenizer.tokenize(sample_text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(f'{sample_text}')
print('='*60)
print(tokens)
print('='*60)
print(ids)

originally gave this a 2 star but I think it only deserves 1. I simply can't wrap my head around the following. picture this, you add a task to your list. the reminder feature is on.you want to change the time on the reminder feature so you click it.
['originally', 'gave', 'this', 'a', '2', 'star', 'but', 'I', 'think', 'it', 'only', 'deserves', '1', '.', 'I', 'simply', 'can', "'", 't', 'wrap', 'my', 'head', 'around', 'the', 'following', '.', 'picture', 'this', ',', 'you', 'add', 'a', 'task', 'to', 'your', 'list', '.', 'the', 'reminder', 'feature', 'is', 'on', '.', 'you', 'want', 'to', 'change', 'the', 'time', 'on', 'the', 'reminder', 'feature', 'so', 'you', 'click', 'it', '.']
[2034, 1522, 1142, 170, 123, 2851, 1133, 146, 1341, 1122, 1178, 18641, 122, 119, 146, 2566, 1169, 112, 189, 10561, 1139, 1246, 1213, 1103, 1378, 119, 3439, 1142, 117, 1128, 5194, 170, 4579, 1106, 1240, 2190, 119, 1103, 15656, 2672, 1110, 1113, 119, 1128, 1328, 1106, 1849, 1103, 1159, 1113, 1103, 15656, 2672, 1177

In [8]:
encoding = tokenizer.encode_plus(
      sample_text,
      add_special_tokens=True,
      max_length=128,
      truncation=True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

c:\Users\Frederico\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
### Train , val and test data 
from sklearn.model_selection import train_test_split

# train_df , test_df  = train_test_split(reviews, test_size = 0.2, shuffle = True)
# val_df , test_df  = train_test_split(test_df, test_size = 0.5, shuffle = True)

X_train, X_rem, y_train, y_rem = train_test_split(reviews, all_ratings, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [10]:
# y_test, y_valid, y_train = y_test.values, y_valid.values, y_train.values

In [11]:
import torch 
from torch.utils.data import Dataset

class AmazonReview(Dataset):
    def __init__(self, review, target, tokenizer, max_len):
        self.review = review
        self.target = target
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.review)
    
    def __getitem__(self,index):
        review = self.review[index]
        encoding = self.tokenizer.encode_plus(
                          review,
                          add_special_tokens=True,
                          max_length=128,
                          truncation=True,
                          return_token_type_ids=False,
                          pad_to_max_length=True,
                          return_attention_mask=True,
                          return_tensors='pt')
        
        return {'review' : review,
                'input_id': encoding['input_ids'].flatten(),
                'attention_mask':encoding['attention_mask'].flatten(),
                'target': torch.tensor(int(self.target[index]))
               }

In [12]:
tokenizer = transformers.BertTokenizer.from_pretrained(MODEL_NAME)
MAX_LEN = 128

training_data = AmazonReview(review = np.array(X_train),
                               target = np.array(y_train),
                               tokenizer = tokenizer,
                               max_len = MAX_LEN)

validation_data = AmazonReview(review = np.array(X_valid),
                               target = np.array(y_valid),
                               tokenizer = tokenizer,
                               max_len = MAX_LEN)

test_data = AmazonReview(review = np.array(X_test),
                               target = np.array(y_test),
                               tokenizer = tokenizer,
                               max_len = MAX_LEN)

In [13]:
print(len(training_data),len(test_data),len(validation_data))

81468 10184 10184


In [14]:
from torch.utils.data import DataLoader
## DataLoader
BATCH_SIZE = 16
train_loader = DataLoader(training_data , batch_size = BATCH_SIZE , shuffle = True)
test_loader = DataLoader(test_data , batch_size = BATCH_SIZE , shuffle = False)
val_loader = DataLoader(validation_data , batch_size = BATCH_SIZE , shuffle = False)

In [15]:
sample_batch = iter(train_loader)
sample_batch = next(sample_batch)
# sample_batch.keys()

c:\Users\Frederico\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
from transformers import BertModel

bert_model = BertModel.from_pretrained(MODEL_NAME)


## testing the output of bert model
x = bert_model(sample_batch['input_id'].view(BATCH_SIZE,MAX_LEN)
               ,sample_batch['attention_mask'].view(BATCH_SIZE,MAX_LEN))

print('Last hidden layer size for input as batch',x[0].shape)
print('Pooled output size for input as batch',x[1].shape)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Last hidden layer size for input as batch torch.Size([16, 128, 768])
Pooled output size for input as batch torch.Size([16, 768])


In [17]:
from torch import nn

class SentimentClassifier(nn.Module):
    def __init__(self,num_classes):
        super(SentimentClassifier,self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.dropout = nn.Dropout(p = 0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size,num_classes)
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self,input_ids , attention_mask):
        temp = self.bert(input_ids,attention_mask) # Here we have added one linear layer on top of 
        pooled_output = temp[1]                    # BERT-base with number of output = 3 
        out = self.dropout(pooled_output)          # 
        out = self.linear(out)
        return out

In [18]:
num_classes = 5
model = SentimentClassifier(num_classes)

sample_linear_output = model(sample_batch['input_id']
                             ,sample_batch['attention_mask'])

s = nn.Softmax(dim=1)
final_out = s(sample_linear_output)
final_out

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[0.5216, 0.0937, 0.2099, 0.1748],
        [0.4046, 0.1596, 0.1546, 0.2812],
        [0.3555, 0.2159, 0.2064, 0.2222],
        [0.3607, 0.2436, 0.1144, 0.2814],
        [0.5020, 0.1255, 0.1332, 0.2392],
        [0.5507, 0.0752, 0.1337, 0.2404],
        [0.5529, 0.0875, 0.1336, 0.2260],
        [0.4299, 0.0878, 0.1895, 0.2928],
        [0.4106, 0.1675, 0.2054, 0.2166],
        [0.3305, 0.1669, 0.2075, 0.2952],
        [0.5174, 0.1074, 0.1614, 0.2138],
        [0.4401, 0.1056, 0.1551, 0.2992],
        [0.4641, 0.1432, 0.1202, 0.2725],
        [0.3599, 0.1470, 0.1968, 0.2963],
        [0.5450, 0.1378, 0.1395, 0.1777],
        [0.4932, 0.1124, 0.2015, 0.1929]], grad_fn=<SoftmaxBackward0>)

In [19]:
## loss and optimizer
learning_rate = 1e-5
EPOCHS = 5
total_steps = len(train_loader) * EPOCHS

criterian = torch.nn.CrossEntropyLoss()

optimizer = transformers.AdamW(params = model.parameters(),lr = learning_rate, correct_bias= False)

c:\Users\Frederico\Anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
# function for training the model given the model, dataset and weight directory where they are saved
import time

def train_model(model, trainloader, testloader, weights_dir, num_epochs=100):
    start=time.time()
    
    error = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(0,num_epochs):

        model.train()  # Put the network in train mode
        for d in trainloader:
        #for i, (review, input_id, attention_mask, target) in enumerate(trainloader):
            #print(d.keys())
            #review, input_id, attention_mask, target = review.to(device), input_id.to(device), attention_mask.to(device), target.to(device)

            input_ids = d['input_id']
            attention_masks = d['attention_mask']
            targets = d['target']

            optimizer.zero_grad()  # Set all currenly stored gradients to zero 

            y_pred = model(input_ids , attention_masks)

            loss = criterian(y_pred, targets)

            loss.backward()

            optimizer.step()

            # Compute relevant metrics

            y_pred_max = torch.argmax(y_pred, dim=1)  # Get the labels with highest output probability

            correct = torch.sum(torch.eq(y_pred_max, target)).item()  # Count how many are equal to the true labels

            elapsed = time.time() - start  # Keep track of how much time has elapsed

            # Show progress every 20 batches 
            if not i % 20:
                print(f'epoch: {epoch}, time: {elapsed:.3f}s, loss: {loss.item():.3f}, train accuracy: {correct / batch_size:.3f}')

        correct_total = 0

        model.eval()  # Put the network in eval mode
        for i, (x_batch, y_batch) in enumerate(val_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)  # Move the data to the device that is used

            y_pred = model(x_batch)
            y_pred_max = torch.argmax(y_pred, dim=1)

            correct_total += torch.sum(torch.eq(y_pred_max, y_batch)).item()

        print(f'Accuracy on the test set: {correct_total / len(validation_data):.3f}')

        # Save weights every 10 epochs
        if epoch%10==0:
            torch.save(model.state_dict(), f"./{weights_dir}/epoch-{epoch}_accuracy-{correct_total/len(validation_data):.3f}.pth")

In [37]:
train_model(model, train_loader, test_loader, 'weights')

c:\Users\Frederico\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


IndexError: Target 5 is out of bounds.

In [21]:
def eval_model(model, data_loader, criterian, BATCH_SIZE ,MAX_LEN, n_examples):
    model.eval()  # to make model in evaluastion model , Dropout will not apply 
    eval_loss = []
    correct_pred = 0
    
    with torch.no_grad():  # calculation of gradient is not required on evaluation
        for d in data_loader:
            input_ids = d['input_id']
            attention_masks = d['attention_mask']
            targets = d['target']

            # forward prop
            predictions = model(input_ids , attention_masks)
            loss = criterian(predictions, targets)
            _,pred_classes = torch.max(predictions, dim=1)

            eval_loss.append(loss.item())
            correct_pred += torch.sum(pred_classes==targets)
    return correct_pred / n_examples , np.mean(eval_loss)

## Evaluation

In [ ]:
from tqdm import tqdm

def get_predictions(model, data_loader):
    model = model.eval()

    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in tqdm(data_loader):

            texts = d["review"]
            input_ids = d["input_id"]
            attention_mask = d["attention_mask"]
            targets = d["target"]

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask)
            
            _, preds = torch.max(outputs, dim=1)

            probs = F.softmax(outputs, dim=1)

            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(probs)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, prediction_probs, real_values

: 

In [ ]:
import torch.nn.functional as F
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model,test_loader)


: 

In [ ]:
from sklearn.metrics import classification_report

class_names = ['negative', 'neutral', 'positive']
print(classification_report(y_test, y_pred, target_names=class_names))

: 

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
    plt.ylabel('True sentiment')
    plt.xlabel('Predicted sentiment');

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

: 